In [23]:
import numpy as np
import matplotlib.pyplot as plt
from pyuvdata import UVData, UVCal, UVFlag, utils
from glob import glob
from astropy import units as u
from astropy import constants as c
from astropy.time import Time
from astropy.coordinates import get_body, SkyCoord, AltAz, EarthLocation, Angle
from copy import deepcopy
import os

In [2]:
JDs = [str(i) for i in np.arange(2457548, 2457556)]
datadir = '/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/'
calfitsdir = '/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/'

In [42]:
%%time
for JD in JDs:
    
    datadir_jd = os.path.join(datadir, JD)
    calfitsdir_jd = os.path.join(calfitsdir, JD)
    outpath_jd = os.path.join(datadir, JD)

    datafiles = sorted(glob(os.path.join(datadir_jd,'zen.*uvcRP.uvh5')))

    for datafile in datafiles: 

        # Get Julian date
        fullpath, filename = os.path.split(datafile)
        tmp = filename.split('.')
        juldate = tmp[1]+'.'+tmp[2]

        calfitsfile = os.path.join(calfitsdir_jd, 'zen.' + juldate +'.HH.uvcRP.calfits')

        uvc = UVCal()
        uvd = UVData()

        print(datafile)
        print(calfitsfile)

        # Read the data and calibration
        uvd.read(datafile)
        uvc.read_calfits(calfitsfile)

        # Just straight up copy the calfits to our new directory; we'll fix it later
        uvc.write_calfits(os.path.join(outpath_jd,'zen.' + juldate +'.uvcRP.calfits'), clobber=True)
        # Make flags specifically ... I think homogenizing and broadcasting can happen later?
        uvf_ant = UVFlag(uvc, mode='flag', copy_flags=True)
        # Write out flags from calfits
        uvf_ant.write(os.path.join(outpath_jd,'flag.'+juldate+'.ant.uvh5'), clobber=True)

        # Write out other flags
        uvf = UVFlag(uvd, mode='flag', copy_flags=True, run_check=True)
        uvf_or = deepcopy(uvf)
        uvf_thresh = deepcopy(uvf)
        uvf_greedy = deepcopy(uvf)

        #uvf_or
        uvf_or.to_waterfall(method='or', keep_pol=False)
        #uvf_thresh
        uvf_thresh.to_waterfall(method='mean', keep_pol=False)
        #uvf_greedy 
        uvf_greedy.to_waterfall(method='or', keep_pol=False)
        time_frac = uvf_greedy.flag_array.mean(axis=1).squeeze()
        freq_frac = uvf_greedy.flag_array.mean(axis=0).squeeze()
        uvf_greedy.flag_array[time_frac>0.33,:,0] = True
        uvf_greedy.flag_array[:,freq_frac>0.10,0] = True

        #read out uvf object
        uvf_or.write(os.path.join(outpath_jd,'flag.'+juldate+'.or.uvh5'), clobber=True)
        uvf_thresh.write(os.path.join(outpath_jd,'flag.'+juldate+'.mean.uvh5'), clobber=True)
        uvf_greedy.write(os.path.join(outpath_jd,'flag.'+juldate+'.greedy.uvh5'), clobber=True)

/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/zen.2457548.16694.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457548/zen.2457548.16694.HH.uvcRP.calfits
File /lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/flag.2457548.16694.ant.uvh5 exists; clobbering
File /lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/flag.2457548.16694.or.uvh5 exists; clobbering
File /lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/flag.2457548.16694.mean.uvh5 exists; clobbering
File /lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/flag.2457548.16694.greedy.uvh5 exists; clobbering
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/zen.2457548.17390.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457548/zen.2457548.17390.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/zen.2457548.18086.uvcRP.uvh5
/lustre/aoc/projects/hera/pl

/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/zen.2457548.44531.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457548/zen.2457548.44531.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/zen.2457548.45227.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457548/zen.2457548.45227.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/zen.2457548.45923.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457548/zen.2457548.45923.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/zen.2457548.46619.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457548/zen.2457548.46619.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457548/zen.2457548.47315.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457548/zen.2457548.47315.HH.uvcRP.calfits
/lustre/aoc/projects

/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.23653.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.23653.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.24349.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.24349.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.25044.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.25044.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.25740.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.25740.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.26436.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.26436.HH.uvcRP.calfits
/lustre/aoc/projects

/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.52882.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.52882.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.53578.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.53578.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.54274.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.54274.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.54970.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.54970.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457549/zen.2457549.55666.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457549/zen.2457549.55666.HH.uvcRP.calfits
/lustre/aoc/projects

/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457550/zen.2457550.32004.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457550/zen.2457550.32004.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457550/zen.2457550.32700.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457550/zen.2457550.32700.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457550/zen.2457550.33396.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457550/zen.2457550.33396.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457550/zen.2457550.34092.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457550/zen.2457550.34092.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457550/zen.2457550.34788.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457550/zen.2457550.34788.HH.uvcRP.calfits
/lustre/aoc/projects

...

/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457553/zen.2457553.57058.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457553/zen.2457553.57058.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457553/zen.2457553.57754.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457553/zen.2457553.57754.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457553/zen.2457553.58450.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457553/zen.2457553.58450.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457553/zen.2457553.59146.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457553/zen.2457553.59146.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457553/zen.2457553.59842.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457553/zen.2457553.59842.HH.uvcRP.calfits
/lustre/aoc/projects

/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457554/zen.2457554.36180.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457554/zen.2457554.36180.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457554/zen.2457554.36876.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457554/zen.2457554.36876.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457554/zen.2457554.37572.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457554/zen.2457554.37572.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457554/zen.2457554.38268.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457554/zen.2457554.38268.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457554/zen.2457554.38964.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457554/zen.2457554.38964.HH.uvcRP.calfits
/lustre/aoc/projects

/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457554/zen.2457554.65410.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457554/zen.2457554.65410.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457554/zen.2457554.66105.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457554/zen.2457554.66105.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457555/zen.2457555.16693.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457555/zen.2457555.16693.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457555/zen.2457555.17389.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457555/zen.2457555.17389.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457555/zen.2457555.18085.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457555/zen.2457555.18085.HH.uvcRP.calfits
/lustre/aoc/projects

/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457555/zen.2457555.44531.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457555/zen.2457555.44531.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457555/zen.2457555.45227.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457555/zen.2457555.45227.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457555/zen.2457555.45923.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457555/zen.2457555.45923.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457555/zen.2457555.46619.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457555/zen.2457555.46619.HH.uvcRP.calfits
/lustre/aoc/projects/hera/jaguirre/HERA19Summer2020/RawData/2457555/zen.2457555.47315.uvcRP.uvh5
/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457555/zen.2457555.47315.HH.uvcRP.calfits
/lustre/aoc/projects

FileNotFoundError: [Errno 2] No such file or directory: '/lustre/aoc/projects/hera/plaplant/HERA19Golden/RawData/2457555/zen.2457555.61930.HH.uvcRP.calfits'